## Taming the LIDAR Beast, Part 1

by Dr. Michael Flaxman, Geodesign Technologies<p>
version 1: July 2018

This notebook demonstrates how to pre-process LIDAR files in common LAS/LAZ formats into CSV files suitable for MapD import.  This includes enrichment of the data with several useful calculated attributes.  A blog post is available giving further background.

In [ ]:
<img src="img/pdal_logo.png",width=60,height=60>

We make extensive use of the excellent open source "point data abstraction library" or PDAL.  For full documentation of this tool, please see <a href='https://pdal.io/index.html'>PDAL docs</a>

### Directory Setup

LIDAR files typically come in huge sets of tiled files, so to keep things organized, we'll start by creating a directory called 'laz' to contain all of our raw original data, and an output directory called 'csv' for outputs.  (Keep in mind that our intermediate CSVs here could get big quick, and that you'll want them local to your MapD server instance if possible for speed.)

In [ ]:
#where are we?
pwd_slist = !pwd
cwd = pwd_slist[0]

In [ ]:
# change paths here if needed, default puts everything relative to script directory

input_dir = os.path.join(cwd, "laz") # read input files from here (change as useful)
output_dir = os.path.join(cwd, "csv") # put results in csv folder parallel to our laz folder

# sanity check
if not os.path.exists(input_dir):
    print('Cannot find input directory {}'.format(input_dir))

if not os.path.exists(output_dir):
    print('Cannot find output directory {}'.format(output_dir))

In [ ]:
import os
if not os.path.exists(input_dir):
    !mkdir {input_dir}
if not os.path.exists(output_dir):
    !mkdir {output_dir}

### Obtaining LIDAR Data

If you'd like to follow along with an example set, this tutorial uses NOAA data from 2016 for Jekyll Island, Geogia.  We've put the URLs pointing to this data into a small text file called <a href='https://github.com/mapd/community-geo-demos/blob/master/mapd_jekyll_island_lidar_tiles.txt'>mapd_jekyll_island_lidar_tiles.txt</a>  <p>Download it, and place in your input directory.

In [ ]:
# change our current working directory to the inputs for simplicity
%cd {input_dir}

In [ ]:
# it should look something like this:
!head -3 mapd_jekyll_island_lidar_tiles.txt

If you want to try this on your own data, just put the LAZ files in the same location.  Go ahead and grab all of <a href='https://download.kortforsyningen.dk/content/geodataprodukter?field_korttype_tid_1=440&field_aktualitet_tid=435&field_datastruktur_tid=All&field_scheme_tid=All'>Denmark</a> if you want... 

Same code and ideas apply (but sorry, no Danish language support)!

In [ ]:
# use xargs to parallel download lidar tiles specified in the text file
# the -P arguement is for number of processors, so adjust accordingly
!xargs -i -P 4 wget '{}' -P {output_dir} < mapd_jekyll_island_lidar_tiles.txt

### Setup and test required libraries for this script

In [ ]:
try:
    #import pdal - not needed, since using commandline tools
    import geopandas as gpd
    import pandas as pd

    # standard system libraries
    import os, glob
    import sys, traceback

    print('Imports successful')
except ImportError:
    print('Missing a required library')

In [ ]:
# Use with due dilligence... performs Anaconda installs of any missing packages
# otherwise reports versions of packages currently installed

required_python_packages = ('pdal','geopandas', 'pymapd')
        
for package in required_python_packages:
    # presumes you have Anaconda package manager installed, if not: try pip
    conda_list = !conda list {package}
    if package in conda_list[3]:
        print('Found existing {} install: \n{}'.format(package, conda_list[3]))
    else:
        print('Could not find package {} in Anaconda, installing... '.format(package))
        !conda install {package}

### Convert LIDAR into CSV

Note: CSV is used, despite its data volume because PDAL 1.7.2's binary shapefile exporter doesn't currently write attributes.

### Process LIDAR into CSV

The most basic PDAL parameters are input and output paths, specified using -i and -o

Because CSV is a verbose file format, it is useful to minimize the number of columns converted and to keep their text representation reflective of actual precision. <p> PDAL has a field specification syntax, which is [fieldname]:[decimal_places]<p>We feed that to the writers.text.order parameter, along with the option keep_unspecified="false". <p>Note that the default precision is 3 decimal places.  This is wasteful for some attributes, but too small for latitude and longitude! 

In [ ]:
fieldspec = 'X:7,Y:7,' # keep all 7 decimal places we have of precision for longitude, latitude
fieldspec += 'Z:2,' # our elevations only appear to have 2 places of precision
fieldspec += 'Intensity:0,ReturnNumber:0,NumberOfReturns:0,Classification:0,' #integers or integer codes
fieldspec += 'HeightAboveGround:2' # a newly-computed attribute, with 2 decimal places

fieldspec

Note: This combination of options makes sense for most LIDAR files.  But if using your own data, you might want to try the 'PDAL info' tool to check if other interesting attributes exist, such as RGB color channel information.  This is vendor-specific.

We have added three interesting and common computational filters to our converter below.  These are further described in PDAL's documentation, along with nearly a dozen others.  <ul><li>The 'smurf' filter applies a sophisticated ground-finding algorithm to distinguish ground surface points from others.  <li>The 'hag' filter then computes a 'height above ground' attribute which is very useful in distinguishing features.  <li>The Morton filter re-orders the points from acquisition flightline order to a scheme in which points earlier in the file provide a spatially-balanced sample of the whole file.  This is useful in testing and developing SQL queries in that you can 'limit' the queries easily but still get distributed data points.</ul>

In [ ]:
# specify path to laz files and wildcard
filespec = "{}/*.laz".format(input_dir)

for counter, file in enumerate(glob.glob(filespec)):
    outfile = file.replace('.laz','.csv')
    outpath = outfile.replace(input_dir, output_dir)
    print('Working on file {}: {}'.format(counter, file))
    !pdal translate -i {file} -o {outpath} \
        -f filters.smrf -f filters.hag -f filters.mortonorder \
        --writers.text.keep_unspecified="false" --writers.text.order={fieldspec} 

In [ ]:
# if this worked, your output directory should be getting big
!ls {output_dir}/*.csv

This scriptlet sequentially reads in CSV LIDAR files created by PDAL, and converts the file-level implicit row number from its index into an explicit attribute.  This is done before merging files so as to preserve the Morton numbers.

In [ ]:
# Optional step:  preserve file-level Morton order attributes

# read in uncompresssed csv files, add a first column with sequence number
# named explictly as Morton, so that this persists after combining tiled csvs

file_spec = "*.csv"
csv_count = len(glob.glob(file_spec))

for counter, file in enumerate(glob.glob(file_spec)):
    print('Reading file {} of {}'.format(counter +1, csv_count))
    csv = pd.read_csv(file)
    
    print('Computing morton index for file {} of {}'.format(counter +1, csv_count))
    csv['Morton'] = csv.index
    
    print('Writing file {} of {}'.format(counter +1, csv_count))
    csv.to_csv(file.replace('.csv','_morton.csv'),index=False)
print('Done')

In [ ]:
# Optional step 2:  convert files to binary shapefile format
# (these cannot be appended in MapD 4.0.0, but will be in next release)
# Note: each individual shapefile is limited to 2G max size

file_spec = os.path.join(output_dir, "*_morton.csv")
print('Searching for CSV LIDAR files in directory {}'.format(file_spec))
csv_count = len(glob.glob(file_spec))

from shapely.geometry import Point

for counter, file in enumerate(glob.glob(file_spec)):
    print('Importing file {} of {}'.format(counter +1, csv_count))
    csv = pd.read_csv(file)
    
    print('Creating point geometry column')
    csv['geometry'] = csv.apply(lambda p: Point(p.X, p.Y), axis=1)
    
    print('Converting pandas dataframe to geopandas')
    geocsv = gpd.GeoDataFrame(csv)

    print('Computing morton index for file {} of {}'.format(counter +1, csv_count))
    geocsv['Morton'] = geocsv.index
    
    out_file_name = file.replace('.csv','.shp')
    print('Writing file {} of {}: {}'.format(counter +1, csv_count, out_file_name))
    geocsv.to_file(out_file_name)
print('Done')

Importing file 5 of 33
Creating point geometry column
Converting pandas dataframe to geopandas
Computing morton index for file 5 of 33
Writing file 5 of 33: /home/mapdadmin/demo/jekyll/csv/2016_PostMatthew_GA_17RMQ6043_morton.shp
Importing file 6 of 33
Creating point geometry column


### Generate MapD LIDAR Dataset from CSVs

OK, now  we have the data we need, and nothing we don't, all organized in a format that MapD 4.0 can read.  Let's use the pymapd library to import the data.  This is currently configured to use a local MapD install with default database, username and password, but these can easily be adjusted.

In [ ]:
import pymapd
print(pymapd.__version__)

In [ ]:
from pymapd import connect

dbname = 'mapd'
user = 'mapd'
host = 'localhost'
password = 'HyperInteractive!'
mapd_con = connect(user=user, password=password, host=host, dbname=dbname)

In [ ]:
mapd_cursor = mapd_con.cursor()
mapd_cursor

In [ ]:
table_name = 'jekyll_lidar_2016v4'
query = "CREATE TABLE IF NOT EXISTS {} ".format(table_name)
query += '(pid BIGINT, p GEOMETRY(POINT, 4326), z FLOAT, '
query += 'Intensity FLOAT, ReturnNumber INTEGER, NumberOfReturns INTEGER, '
query += 'Classification SMALLINT, Morton BIGINT); 
query'

In [ ]:
import sys,traceback

try:
    print('Executing query: {}'.format(query))
    results = mapd_cursor.execute(query)
except:
    print('Exception executing query')
    a,b,c = sys.exc_info()
    for d in traceback.format_exception(a,b,c) :
       print (d)

In [ ]:
#import each file in sequence, with MapD appending
file_spec = os.path.join(output_dir, "*_morton.csv")
print('Searching for CSV LIDAR files in directory {}'.format(file_spec))
csv_count = len(glob.glob(file_spec))
csv_count

for counter, file in enumerate(glob.glob(file_spec)):
    print('Importing file {} of {}'.format(counter +1, csv_count))
    query = "COPY {} FROM '{}'; ".format(table_name,file)
    try:
        print('Executing query: {}'.format(query))
        results = mapd_cursor.execute(query)
    except:
        print('Exception executing query')
        a,b,c = sys.exc_info()
        for d in traceback.format_exception(a,b,c) :
           print (d)   

In [ ]:
# clean up intermediate files here if desired
# rm -rf {output_dir}/*.csv